In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from tinyllava.model.builder import load_pretrained_model
from tinyllava.mm_utils import get_model_name_from_path

import torch.nn as nn

model_path = "bczhou/TinyLLaVA-2.0B"

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=get_model_name_from_path(model_path)
)

#from serve import infer

#result = infer(tokenizer, model, image_processor, "What is cute about this image?", image_file="Cute_dog.jpg")
#summary(model, (context_len))
print(model)
#print(result)

# model.lm_head = Identity()
# result = infer_no_lm(tokenizer, model, image_processor, "What is cute about this image?", image_file="Cute_dog.jpg")
# #summary(model, (context_len))
# print(model)
# print(result.shape)



/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bczhou/TinyLLaVA-2.0B were not used when initializing TinyLlavaStablelmForCausalLM: ['model.vision_tower.vision_tower.vision_model.embeddings.patch_embedding.bias', 'model.vision_tower.vision_tower.vision_model.embeddings.patch_embedding.weight', 'model.vision_tower.vision_tower.vision_model.embeddings.position_embedding.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm2.weight', 'model.vision_tower.vision_tower.vision_model.encod

TinyLlavaStablelmForCausalLM(
  (model): TinyLlavaStablelmModel(
    (embed_tokens): Embedding(100289, 2048)
    (layers): ModuleList(
      (0-23): 24 x DecoderLayer(
        (self_attn): Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): MLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, element

In [6]:
from serve import infer_no_lm, infer
from copy import deepcopy
llm = deepcopy(model)

input_text = "Say a long clever sentence"

# out, out_ids = infer(tokenizer, llm, image_processor, input_text, image_file="Cute_dog.jpg")
# print(out_ids.shape)
# out, out_ids = infer(tokenizer, llm, image_processor, input_text, None)
# print(out_ids.shape)

#llm.lm_head = nn.Identity()
# llm.model.mm_projector = nn.Identity()
out = infer_no_lm(tokenizer, llm, image_processor, input_text, image_file="Cute_dog.jpg")
print(out.past_key_values[0][0].shape)
print(out.logits.shape)
print(out.logits[0].shape)
out = infer_no_lm(tokenizer, llm, image_processor, input_text, None)
print(out.past_key_values[0][0].shape)
print(out.logits.shape)
print(out.logits[0].shape)
print(len(out.hidden_states))

# This one is it!
print(out.hidden_states[0].shape)


torch.Size([1, 32, 770, 64])
torch.Size([1, 770, 100289])
torch.Size([770, 100289])
torch.Size([1, 32, 39, 64])
torch.Size([1, 39, 100289])
torch.Size([39, 100289])
25
torch.Size([1, 39, 2048])


In [8]:
print(out.hidden_states[0][0].mean(dim=0).shape)

torch.Size([39])
